# Video Processing in a Sliding Window (Experiment)

This notebook demonstrates reading a video using OpenCV and processing it **a few frames at a time**, keeping a queue of up to `batch_size` frames and processing it. 
Each time we add a new frame, if the queue is full, we drop the oldest frame. This effectively gives us a rolling/sliding window of frames.

A degenerated usage of processing 1 frame per iteration can be achieved by initializing ProcessVideoExperiment with batch_size=1.

In [ ]:
# If you need to install opencv-python, uncomment and run:
# !pip install opencv-python

import numpy as np
import cv2 as cv
from collections import deque
from datetime import datetime
from video_batch_processor import VideoBatchProcessor
from timer import Timer
from metrics_extractor import DEFAULT_FARNEBACK_PARAMS, DEFAULT_LUCAS_KANADE_PARAMS, DEFAULT_TVL1_PARAMS, \
extract_farneback_metric, extract_TVL1_metric, extract_lucas_kanade_metric, extract_metrics, append_metrics

### Set Video capture device, Batch Size, and Timer object
- `batch_size`: how many frames to store in the sliding window.
- `video_capture_object`: the object that provides frames to process. 
                          can be camera/file.
- `timer_object`: the timer object we use to make timestamps

In [ ]:
IS_WEBCAM = False  # toggle: True -> capture from webcam. False -> capture from file.

if IS_WEBCAM:
        video_source = 0
else:  # video source is a video file
        video_source = "Workable Data/Processed/DPH0_Surface_TL.avi"
        
video_capture_object = cv.VideoCapture(video_source)

batch_size = 5  # Number of frames in the sliding window

if not video_capture_object.isOpened():
        if IS_WEBCAM:
                raise IOError(f"Cannot open webcam.")
        else:
                raise IOError(f"Cannot open video file: {video_source}.")

timer_object = Timer()

### Setup Hyper-Parameters
- `super_pixel_dimensions`: a tuple of (x_size, y_size)
- `frame_window_size`: the queue size of the moving window of frames.

In [ ]:
super_pixel_dimensions = (2, 2)  # can be modified
frame_window_size = 5

### Process Function
We'll define a simple function to illustrate what you might do with each batch (window) of frames.
- Here, we just print out how many frames are in the current window.
- You could replace this with custom logic (e.g., saving images, running inference, etc.).

In [ ]:
def process_frame_window(frames: deque):
    current_timestamp = datetime.now()

    if len(frames) < 2:
        raise RuntimeError("frame window size must have at least 2 frames.")
    
    frame1 = frames[0]   # take first frame in frame window
    frame2 = frames[-1]  # take last frame in frame window

    # prepare the metric extractors
    metric_extractors = dict()
    metric_extractors["Lucas-Kanade"] = {
        "kwargs": DEFAULT_LUCAS_KANADE_PARAMS,
        "function": extract_lucas_kanade_metric
    }
    metric_extractors["Farneback"] = {
        "kwargs": DEFAULT_FARNEBACK_PARAMS,
        "function": extract_farneback_metric
    }
    metric_extractors["TVL1"] = {
        "kwargs": DEFAULT_TVL1_PARAMS,
        "function": extract_TVL1_metric
    }
    # TODO: add more metric extractors here

    metrics = extract_metrics(frame1, frame2, metric_extractors)
    
    return metrics

### Setup for Main Loop

In [ ]:
frame_window = deque()  # Init as deque so i can pop left with minimal overhead

for i in range(frame_window_size):
    ret, frame = video_capture_object.read()  # Read a frame

    if not ret:
        print("End of video or error")
        break

    frame_window.append(frame)

timer_object.tick()  # Start the clock

### Main Loop
TODO:

In [ ]:
try:
    frame_count = frame_window_size    
    today = timer_object.start_time
    output_prefix = f'{today.year}-{today.month}-{today.day}'

    if IS_WEBCAM:
        output_suffix = 'time_microsec'
    else:  # is video file
        output_suffix = 'frame_count'
        

    while True:
        ret, frame = video_capture_object.read()  # Read a frame

        if not ret:
            print("End of video or error")
            break

        frame_window.popleft()
        frame_window.append(frame)

        if IS_WEBCAM:  # t-axis is elapsed miliseconds
            # taking the elapsed before, so i can know the time of when the last frame was taken.
            elapsed_miliseconds = timer_object.tock() // 1000  # convert micro to mili
            elapsed_t_units = elapsed_miliseconds
        else:  # If file then the t-axis is frame count
            elapsed_t_units = frame_count

        # NOTE: this is the HEAVY function
        # metrics is now a dictionary of the form: {"varient_name": <metrics dictionary>}
        metrics = process_frame_window(frame_window)

        # log the metrics with the elapsed units
        output_file_path = f'./output/{output_prefix}_{video_source[:15]}_flow_metrics_{output_suffix}.csv'
        append_metrics(output_file_path, metrics, elapsed_t_units)

        # TODO: display updating video and graph

        frame_count += 1

finally:  # release resources
    video_capture_object.release()
    cv.destroyAllWindows()


Finished processing the video.
